In [ ]:
import librosa
import os # библиотека для работы с файлами
import matplotlib.pyplot as plt
import librosa.display as ld
import numpy as np
import pandas as pd
import random

## Предобработка всех файлов директории, создание датафрейма

### Извлечение характеристик из файлов

In [ ]:
n_fft = 8195
sr = 22050
hop_length = n_fft // 2

def spectrum_feature(signal):
    ft = np.abs(librosa.stft(signal[:n_fft], n_fft=n_fft, hop_length=hop_length))
    mean_ft = np.mean(ft, axis=1)  # Усредняем по всем каналам

    return np.mean(mean_ft), np.var(mean_ft)

def db_spectrum_feature(signal):
    X = librosa.stft(signal)
    s = librosa.amplitude_to_db(abs(X))

    return np.mean(s), np.var(s)

def melspectrum_feature(signal):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc = 40, hop_length=512)

    return np.mean(mfccs), np.var(mfccs)

def db_melspectrum_feature(signal):
    melspectrum = librosa.feature.melspectrogram(y=signal, sr=sr, hop_length=512, n_mels=40)
    melspectrum_db = librosa.power_to_db(melspectrum, ref=np.max)  # Преобразуем в децибелы

    return np.mean(melspectrum_db), np.var(melspectrum_db)

def spectral_cent_feature(signal):
    cent = librosa.feature.spectral_centroid(y=signal, sr=sr)

    return np.mean(cent), np.var(cent)

def spectral_rolloff_feature(signal):
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)

    return np.mean(rolloff), np.var(rolloff)

def zero_crossing_feature(signal):
    zrate=librosa.feature.zero_crossing_rate(signal)

    return np.mean(zrate), np.var(zrate)

### Извлечение MFCC по группам

In [ ]:
# Функция для вычисления мел-кепстральных коэффициентов
def compute_mfcc(audio, sr=22050, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Функция для разбиения аудио на фрагменты по 20 секунд
def split_audio(audio, sr=22050, fragment_length=20):
    fragment_samples = fragment_length * sr
    num_fragments = len(audio) // fragment_samples
    fragments = [audio[i*fragment_samples:(i+1)*fragment_samples] for i in range(num_fragments)]
    return fragments

# Функция для разбиения мел-кепстральных коэффициентов на группы по 20 элементов
def split_mfcc(mfccs, group_size=20):
    num_frames = mfccs.shape[1]
    num_groups = num_frames // group_size
    mfcc_groups = [mfccs[:, i*group_size:(i+1)*group_size] for i in range(num_groups)]
    return mfcc_groups

# Функция для обработки отдельного аудиофайла
def process_audio(file_path, max_duration, fragment_length=20, group_size=20):
    try:
        audio, sr = librosa.load(file_path, sr=None, duration=max_duration)
        audio_length = librosa.get_duration(y=audio, sr=sr)
        if audio_length < fragment_length:
            return None
        fragments = split_audio(audio, sr, fragment_length)
        data = []
        filename = os.path.splitext(os.path.basename(file_path))[0]
        for i, fragment in enumerate(fragments):
            mfccs = compute_mfcc(fragment, sr)
            mfcc_groups = split_mfcc(mfccs, group_size)
            fragment_data = {'filename': f"{filename}_{i+1}"}
            for j, group in enumerate(mfcc_groups):
                mean = np.mean(group, axis=1)
                std_dev = np.std(group, axis=1)
                for k in range(mean.shape[0]):
                    fragment_data[f'MFCC_Mean_{(j*group_size)+k+1}'] = mean[k]
                    fragment_data[f'MFCC_StdDev_{(j*group_size)+k+1}'] = std_dev[k]
            data.append(fragment_data)
        return data
    except Exception as e:
        print(f"Error processing audio {file_path}: {e}")
        return None



# Функция для обработки всех аудиофайлов в директории
def process_directory(directory_path, fragment_length=20, group_size=20, max_duration=60):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.mp3'):
            file_path = os.path.join(directory_path, filename)
            result = process_audio(file_path, max_duration, fragment_length, group_size)
            if result is not None:
                data.extend(result)
    return data

# Путь к директории с аудиофайлами
directory_path = 'songs'
# Обработка всех аудиофайлов в директории
data = process_directory(directory_path)

csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Создание датафрейма из полученных данных
df = pd.DataFrame(data)

# Вывод датафрейма
print(df)
csv_path = 'songs_features.csv'
df.to_csv(csv_path, index=False)

In [2]:
import numpy as np
import pandas as pd
import os
import librosa

# Функция для вычисления мел-кепстральных коэффициентов
def compute_mfcc(audio, sr=22050, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Функция для разбиения аудио на фрагменты по 20 секунд с перекрытием в 5 секунд
def split_audio(audio, sr=22050, fragment_length=20, overlap=5):
    fragment_samples = fragment_length * sr
    overlap_samples = overlap * sr
    start_samples = 0
    fragments = []
    while start_samples + fragment_samples <= len(audio):
        fragments.append(audio[start_samples:start_samples+fragment_samples])
        start_samples += overlap_samples
    return fragments

# Функция для обработки отдельного аудиофайла
def process_audio(file_path, max_duration, fragment_length=20, overlap=5):
    try:
        audio, sr = librosa.load(file_path, sr=None, duration=max_duration)
        audio_length = librosa.get_duration(y=audio, sr=sr)
        if audio_length < fragment_length:
            return None
        fragments = split_audio(audio, sr, fragment_length, overlap)
        data = []
        filename = os.path.splitext(os.path.basename(file_path))[0]
        for i, fragment in enumerate(fragments):
            mfccs = compute_mfcc(fragment, sr)
            mean = np.mean(mfccs, axis=1)
            std_dev = np.std(mfccs, axis=1)
            fragment_data = {'filename': f"{filename}_{i+1}"}
            for j in range(mean.shape[0]):
                fragment_data[f'MFCC_Mean_{j+1}'] = mean[j]
                fragment_data[f'MFCC_StdDev_{j+1}'] = std_dev[j]
            data.append(fragment_data)
        return data
    except Exception as e:
        print(f"Error processing audio {file_path}: {e}")
        return None

# Функция для обработки всех аудиофайлов в директории
def process_directory(directory_path, fragment_length=20, overlap=5, max_duration=60):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.mp3'):
            file_path = os.path.join(directory_path, filename)
            result = process_audio(file_path, max_duration, fragment_length, overlap)
            if result is not None:
                data.extend(result)
    return data

# Путь к директории с аудиофайлами
directory_path = 'songs'
# Обработка всех аудиофайлов в директории
data = process_directory(directory_path)

csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Создание датафрейма из полученных данных
df = pd.DataFrame(data)

# Вывод датафрейма
print(df)

csv_path = 'songs_features.csv'
df.to_csv(csv_path, index=False)

Файл 'songs_features.csv' очищен успешно.
                                        filename  MFCC_Mean_1  MFCC_StdDev_1  \
0    7 Seconds -- Youssou N'Dour, Neneh Cherry_1  -220.852127     114.887589   
1    7 Seconds -- Youssou N'Dour, Neneh Cherry_2  -177.186890      53.960754   
2    7 Seconds -- Youssou N'Dour, Neneh Cherry_3  -175.859924      55.398659   
3    7 Seconds -- Youssou N'Dour, Neneh Cherry_4  -168.870377      55.242714   
4    7 Seconds -- Youssou N'Dour, Neneh Cherry_5  -158.136078      53.203526   
..                                           ...          ...            ...   
121              Bag Raiders -- Shooting stars_5  -153.237335      99.889580   
122              Bag Raiders -- Shooting stars_6  -112.452782      53.073341   
123              Bag Raiders -- Shooting stars_7  -105.967949      51.869789   
124              Bag Raiders -- Shooting stars_8   -99.754913      50.877182   
125              Bag Raiders -- Shooting stars_9   -92.138924      49.087082  

In [ ]:
df.columns

In [3]:
len_max = max([len(col) for col in df.columns])
for col in df.columns:
    print(f"{col:<{len_max}} labels: {len(df[col].unique())}")

filename       labels: 126
MFCC_Mean_1    labels: 126
MFCC_StdDev_1  labels: 126
MFCC_Mean_2    labels: 126
MFCC_StdDev_2  labels: 126
MFCC_Mean_3    labels: 126
MFCC_StdDev_3  labels: 126
MFCC_Mean_4    labels: 126
MFCC_StdDev_4  labels: 126
MFCC_Mean_5    labels: 126
MFCC_StdDev_5  labels: 126
MFCC_Mean_6    labels: 126
MFCC_StdDev_6  labels: 126
MFCC_Mean_7    labels: 126
MFCC_StdDev_7  labels: 126
MFCC_Mean_8    labels: 126
MFCC_StdDev_8  labels: 126
MFCC_Mean_9    labels: 126
MFCC_StdDev_9  labels: 126
MFCC_Mean_10   labels: 126
MFCC_StdDev_10 labels: 126
MFCC_Mean_11   labels: 126
MFCC_StdDev_11 labels: 126
MFCC_Mean_12   labels: 126
MFCC_StdDev_12 labels: 126
MFCC_Mean_13   labels: 126
MFCC_StdDev_13 labels: 126


### Обработка аудиофайлов целиком

In [ ]:
csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")


# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого DataFrame с колонкой "filename"
df = pd.DataFrame({'filename': file_names})

# Добавление признаков в DataFrame
for func_name, func in [('spectrum_feature', spectrum_feature),
                        ('db_spectrum_feature', db_spectrum_feature),
                        ('melspectrum_feature', melspectrum_feature),
                        ('db_melspectrum_feature', db_melspectrum_feature),
                        ('spectral_cent_feature', spectral_cent_feature),
                        ('spectral_rolloff_feature', spectral_rolloff_feature),
                        ('zero_crossing_feature', zero_crossing_feature)]:
    # Применение функции к каждому файлу и добавление результатов в DataFrame
    mean_var = df['filename'].apply(lambda x: pd.Series(func(librosa.load(songs_dir + x, sr=22050)[0]))) # решить с каналом
    df[[f'{func_name}_mean', f'{func_name}_var']] = mean_var

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)

### Обработка дискретизированного аудиофайла

In [ ]:
n_fft = 8195
sr = 22050
hop_length = n_fft // 2
segment_duration = 20  # Длительность каждого сегмента в секундах
segment_samples = segment_duration * sr  # Количество отсчетов в каждом сегменте

def segment_audio(signal, sr, segment_duration):
    # Разбиваем аудиозапись на сегменты по segment_duration секунд
    segments = []
    for start_sample in range(0, len(signal), segment_duration * sr):
        segment = signal[start_sample:start_sample + segment_duration * sr]
        # Если сегмент короче ожидаемой длительности, дополним его нулями
        if len(segment) < segment_duration * sr:
            segment = np.concatenate([segment, np.zeros(segment_duration * sr - len(segment))])
        segments.append(segment)
    return segments

def extract_features(signal):
    features = {}
    features['spectrum_mean'], features['spectrum_var'] = spectrum_feature(signal)
    features['db_spectrum_mean'], features['db_spectrum_var'] = db_spectrum_feature(signal)
    features['melspectrum_mean'], features['melspectrum_var'] = melspectrum_feature(signal)
    features['db_melspectrum_mean'], features['db_melspectrum_var'] = db_melspectrum_feature(signal)
    features['spectral_cent_mean'], features['spectral_cent_var'] = spectral_cent_feature(signal)
    features['spectral_rolloff_mean'], features['spectral_rolloff_var'] = spectral_rolloff_feature(signal)
    features['zero_crossing_mean'], features['zero_crossing_var'] = zero_crossing_feature(signal)
    return features


csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого списка для хранения словарей признаков
segments_data = []

# Добавление признаков в DataFrame для каждого сегмента каждой композиции

for filename in file_names:
    file_path = os.path.join(songs_dir, filename)
    signal, sr = librosa.load(file_path, sr=sr)
    segments = segment_audio(signal, sr, segment_duration)
    for i, segment in enumerate(segments):
        # Обрезаем тишину в начале и в конце сегмента
        segment, _ = librosa.effects.trim(segment) # эта строка убирается, если тишину необходимо оставить
        features = extract_features(segment)
        segment_data = {'filename': f"{filename[:-4]}_{i+1}", **features}
        segments_data.append(segment_data)

# Создание DataFrame из списка словарей
df = pd.DataFrame(segments_data)

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)


In [ ]:
import numpy as np
import os
import pandas as pd
import librosa
from scipy.stats import moment

n_fft = 8195
sr = 22050
hop_length = n_fft // 2
segment_duration = 20  # Длительность каждого сегмента в секундах
segment_samples = segment_duration * sr  # Количество отсчетов в каждом сегменте

def segment_audio(signal, sr, segment_duration, overlap):
    # Разбиваем аудиозапись на сегменты по segment_duration секунд с перекрытием overlap
    segments = []
    start = 0
    while start + segment_duration <= len(signal) / sr:
        segment = signal[int(start * sr): int((start + segment_duration) * sr)]
        # Если сегмент короче ожидаемой длительности, дополним его нулями
        if len(segment) < segment_duration * sr:
            segment = np.concatenate([segment, np.zeros(segment_duration * sr - len(segment))])
        segments.append(segment)
        start += overlap
    return segments

def extract_features(signal):
    features = {}
    features['spectrum_mean'], features['spectrum_var'] = spectrum_feature(signal)
    features['db_spectrum_mean'], features['db_spectrum_var'] = db_spectrum_feature(signal)
    features['melspectrum_mean'], features['melspectrum_var'] = melspectrum_feature(signal)
    features['db_melspectrum_mean'], features['db_melspectrum_var'] = db_melspectrum_feature(signal)
    features['spectral_cent_mean'], features['spectral_cent_var'] = spectral_cent_feature(signal)
    features['spectral_rolloff_mean'], features['spectral_rolloff_var'] = spectral_rolloff_feature(signal)
    features['zero_crossing_mean'], features['zero_crossing_var'] = zero_crossing_feature(signal)
    return features


csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого списка для хранения словарей признаков
segments_data = []

# Добавление признаков в DataFrame для каждого сегмента каждой композиции

for filename in file_names:
    file_path = os.path.join(songs_dir, filename)
    signal, sr = librosa.load(file_path, sr=sr)
    segments = segment_audio(signal, sr, segment_duration, segment_duration - 5)  # Изменение с перекрытием
    for i, segment in enumerate(segments):
        # Обрезаем тишину в начале и в конце сегмента
        segment, _ = librosa.effects.trim(segment) # эта строка убирается, если тишину необходимо оставить
        features = extract_features(segment)
        segment_data = {'filename': f"{filename[:-4]}_{i+1}", **features}
        segments_data.append(segment_data)

# Создание DataFrame из списка словарей
df = pd.DataFrame(segments_data)

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)


## Тестирование

In [ ]:
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/Timbaland_-_Apologize_47972715.mp3'
signal = librosa.load(filename_new, sr=22050)[0] #, duration=20
spectrum_mean, spectrum_var = spectrum_feature(signal)
db_spectrum_mean, db_spectrum_var = db_spectrum_feature(signal)
melspectrum_mean, melspectrum_var = melspectrum_feature(signal)
db_melspectrum_mean, db_melspectrum_var = db_melspectrum_feature(signal)
spectral_cent_mean, spectral_cent_var = spectral_cent_feature(signal)
spectral_rolloff_mean, spectral_rolloff_var = spectral_rolloff_feature(signal)
zero_crossing_mean, zero_crossing_var = zero_crossing_feature(signal)

new_features = [spectrum_mean, spectrum_var, db_spectrum_mean, db_spectrum_var, melspectrum_mean, melspectrum_var, db_melspectrum_mean, db_melspectrum_var, spectral_cent_mean, spectral_cent_var, spectral_rolloff_mean, spectral_rolloff_var, zero_crossing_mean, zero_crossing_var]
new_features = np.array(new_features).reshape(1, -1)



X = df.drop(columns=['filename'])
print(X)
print('Features:')
print(new_features)

### Поиск по дистанциям

In [ ]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])


### Поиск по сумме дистанций фрагментов (только для дискретизованных)

In [ ]:
# Создайте словарь для хранения сумм расстояний по композициям
sum_distances = {}

# Проход по каждой строке DataFrame X (признаки каждого фрагмента)
for index, row in df.iterrows():
    # Преобразуйте индекс в строку и получите название композиции из индекса строки
    composition_name = row['filename'].split('_')[0]
    
    # Если композиция еще не добавлена в словарь, добавьте ее
    if composition_name not in sum_distances:
        sum_distances[composition_name] = 0
    
    # Вычислите расстояние между текущим фрагментом и новой композицией
    distance = np.linalg.norm(X.values - new_features)
    
    # Добавьте расстояние к сумме расстояний для данной композиции
    sum_distances[composition_name] += distance

# Преобразуйте словарь в DataFrame для удобства обработки
sum_distances_df = pd.DataFrame(list(sum_distances.items()), columns=['Composition', 'Sum Distance'])

# Найдите наиболее похожие композиции на основе сумм расстояний
closest_compositions = sum_distances_df.nsmallest(11, 'Sum Distance')

# Выведите результаты
print("Наиболее похожие композиции:")
for index, row in closest_compositions.iterrows():
    print(f"{row['Composition']}, Сумма расстояний: {row['Sum Distance']}")


### Поиск для MFCC

### Загрузка файла

In [ ]:
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/Timbaland_feat_OneRepublic_-_Apologize_NORTKASH_Zusebi_Remix_73927193.mp3'
result = process_audio(filename_new, 20, fragment_length=20, overlap=5)
result = list(result[0].values())[1:]
new_features = np.array(result).reshape(1, -1)

X = df.drop(columns=['filename'])
print('Features:')
print(new_features)

In [ ]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Загрузка данных
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/7 Seconds from middle small.mp3'
result = process_audio(filename_new, 20, fragment_length=20, overlap=5)
result = list(result[0].values())[1:]
new_features = np.array(result).reshape(1, -1)

# Исключаем столбец с именем файла
X = df.drop(columns=['filename'])

# Создание модели kNN
k = 11  # количество соседей
knn = NearestNeighbors(n_neighbors=k)
knn.fit(X)

# Поиск ближайших соседей
distances, indices = knn.kneighbors(new_features)

# Получение информации о ближайших соседях
closest_files = df.iloc[indices[0]]['filename']
closest_features = X.iloc[indices[0]].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_files)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[0][i])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Загрузка данных
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/7 Seconds from middle small.mp3'
result = process_audio(filename_new, 20, fragment_length=20, overlap=5)
result = list(result[0].values())[1:]
new_features = np.array(result).reshape(1, -1)

X = df.drop(columns=['filename'])

# Вычислите косинусное сходство между наборами признаков
similarities = cosine_similarity(X, new_features)

# Найдите индексы строк с максимальным косинусным сходством (включая самый близкий)
closest_indices = np.argsort(similarities, axis=0)[-11:].flatten()

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Косинусное сходство:", similarities[closest_indices[i]][0])

In [ ]:
# Создайте словарь для хранения сумм расстояний по композициям
sum_distances = {}

# Проход по каждой строке DataFrame X (признаки каждого фрагмента)
for index, row in df.iterrows():
    # Преобразуйте индекс в строку и получите название композиции из индекса строки
    composition_name = row['filename'].split('_')[0]
    
    # Если композиция еще не добавлена в словарь, добавьте ее
    if composition_name not in sum_distances:
        sum_distances[composition_name] = 0
    
    # Вычислите евклидово расстояние между текущим фрагментом и новой композицией
    distance = np.linalg.norm(row.values[1:] - new_features)
    
    # Добавьте расстояние к сумме расстояний для данной композиции
    sum_distances[composition_name] += distance

# Преобразуйте словарь в DataFrame для удобства обработки
sum_distances_df = pd.DataFrame(list(sum_distances.items()), columns=['Composition', 'Sum Distance'])

closest_compositions = sum_distances_df.nsmallest(11, 'Sum Distance')

print("Наиболее похожие композиции:")
for index, row in closest_compositions.iterrows():
    print(f"{row['Composition']}, Сумма расстояний: {row['Sum Distance']}")


## Предобработка всех файлов директории, создание датафрейма

In [34]:
n_fft = 8195
sr = 22050
hop_length = n_fft // 2

def spectrum_feature(signal):
    ft = np.abs(librosa.stft(signal[:n_fft], n_fft=n_fft, hop_length=hop_length))
    mean_ft = np.mean(ft, axis=1)  # Усредняем по всем каналам

    return np.mean(mean_ft), np.var(mean_ft)

def db_spectrum_feature(signal):
    X = librosa.stft(signal)
    s = librosa.amplitude_to_db(abs(X))

    return np.mean(s), np.var(s)

def melspectrum_feature(signal):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc = 40, hop_length=512)

    return np.mean(mfccs), np.var(mfccs)

def db_melspectrum_feature(signal):
    melspectrum = librosa.feature.melspectrogram(y=signal, sr=sr, hop_length=512, n_mels=40)
    melspectrum_db = librosa.power_to_db(melspectrum, ref=np.max)  # Преобразуем в децибелы

    return np.mean(melspectrum_db), np.var(melspectrum_db)

def spectral_cent_feature(signal):
    cent = librosa.feature.spectral_centroid(y=signal, sr=sr)

    return np.mean(cent), np.var(cent)

def spectral_rolloff_feature(signal):
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)

    return np.mean(rolloff), np.var(rolloff)

def zero_crossing_feature(signal):
    zrate=librosa.feature.zero_crossing_rate(signal)

    return np.mean(zrate), np.var(zrate)

### Обработка аудиофайлов целиком

In [6]:
# Функция для вычисления мел-кепстральных коэффициентов
def compute_mfcc(audio, sr=22050, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Функция для разбиения аудио на фрагменты по 20 секунд
def split_audio(audio, sr=22050, fragment_length=20):
    fragment_samples = fragment_length * sr
    num_fragments = len(audio) // fragment_samples
    fragments = [audio[i*fragment_samples:(i+1)*fragment_samples] for i in range(num_fragments)]
    return fragments

# Функция для разбиения мел-кепстральных коэффициентов на группы по 20 элементов
def split_mfcc(mfccs, group_size=20):
    num_frames = mfccs.shape[1]
    num_groups = num_frames // group_size
    mfcc_groups = [mfccs[:, i*group_size:(i+1)*group_size] for i in range(num_groups)]
    return mfcc_groups

# Функция для обработки отдельного аудиофайла
def process_audio(file_path, max_duration, fragment_length=20, group_size=20):
    try:
        audio, sr = librosa.load(file_path, sr=None, duration=max_duration)
        audio_length = librosa.get_duration(y=audio, sr=sr)
        if audio_length < fragment_length:
            return None
        fragments = split_audio(audio, sr, fragment_length)
        data = []
        filename = os.path.splitext(os.path.basename(file_path))[0]
        for i, fragment in enumerate(fragments):
            mfccs = compute_mfcc(fragment, sr)
            mfcc_groups = split_mfcc(mfccs, group_size)
            fragment_data = {'filename': f"{filename}_{i+1}"}
            for j, group in enumerate(mfcc_groups):
                mean = np.mean(group, axis=1)
                std_dev = np.std(group, axis=1)
                for k in range(mean.shape[0]):
                    fragment_data[f'MFCC_Mean_{(j*group_size)+k+1}'] = mean[k]
                    fragment_data[f'MFCC_StdDev_{(j*group_size)+k+1}'] = std_dev[k]
            data.append(fragment_data)
        return data
    except Exception as e:
        print(f"Error processing audio {file_path}: {e}")
        return None



# Функция для обработки всех аудиофайлов в директории
def process_directory(directory_path, fragment_length=20, group_size=20, max_duration=60):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.mp3'):
            file_path = os.path.join(directory_path, filename)
            result = process_audio(file_path, max_duration, fragment_length, group_size)
            if result is not None:
                data.extend(result)
    return data

# Путь к директории с аудиофайлами
directory_path = 'songs'
# Обработка всех аудиофайлов в директории
data = process_directory(directory_path)

csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Создание датафрейма из полученных данных
df = pd.DataFrame(data)

# Вывод датафрейма
print(df)
csv_path = 'songs_features.csv'
df.to_csv(csv_path, index=False)

Файл 'songs_features.csv' очищен успешно.
                                             filename  MFCC_Mean_1  \
0         7 Seconds -- Youssou N'Dour, Neneh Cherry_1  -551.628662   
1         7 Seconds -- Youssou N'Dour, Neneh Cherry_2  -140.451599   
2         7 Seconds -- Youssou N'Dour, Neneh Cherry_3  -170.317963   
3                                a-ha -- Take On Me_1  -540.525757   
4                                a-ha -- Take On Me_2  -126.084801   
5                                a-ha -- Take On Me_3   -64.118607   
6                       ABBA -- Money, Money, Money_1  -543.826843   
7                       ABBA -- Money, Money, Money_2  -212.842972   
8                       ABBA -- Money, Money, Money_3  -137.955460   
9                   ABBA -- The Winner Takes It All_1  -698.643555   
10                  ABBA -- The Winner Takes It All_2  -212.278900   
11                  ABBA -- The Winner Takes It All_3  -301.265747   
12                          All The Things She S

In [40]:
df.columns

Index(['filename', 'spectrum_feature_mean', 'spectrum_feature_var',
       'db_spectrum_feature_mean', 'db_spectrum_feature_var',
       'melspectrum_feature_mean', 'melspectrum_feature_var',
       'db_melspectrum_feature_mean', 'db_melspectrum_feature_var',
       'spectral_cent_feature_mean', 'spectral_cent_feature_var',
       'spectral_rolloff_feature_mean', 'spectral_rolloff_feature_var',
       'zero_crossing_feature_mean', 'zero_crossing_feature_var'],
      dtype='object')

### Обработка дискретизированного аудиофайла

In [ ]:
import numpy as np
import os
import pandas as pd
import librosa
from scipy.stats import moment

n_fft = 8195
sr = 22050
hop_length = n_fft // 2
segment_duration = 20  # Длительность каждого сегмента в секундах
segment_samples = segment_duration * sr  # Количество отсчетов в каждом сегменте

def segment_audio(signal, sr, segment_duration, overlap):
    # Разбиваем аудиозапись на сегменты по segment_duration секунд с перекрытием overlap
    segments = []
    start = 0
    while start + segment_duration <= len(signal) / sr:
        segment = signal[int(start * sr): int((start + segment_duration) * sr)]
        # Если сегмент короче ожидаемой длительности, дополним его нулями
        if len(segment) < segment_duration * sr:
            segment = np.concatenate([segment, np.zeros(segment_duration * sr - len(segment))])
        segments.append(segment)
        start += overlap
    return segments

def extract_features(signal):
    features = {}
    features['spectrum_mean'], features['spectrum_var'] = spectrum_feature(signal)
    features['db_spectrum_mean'], features['db_spectrum_var'] = db_spectrum_feature(signal)
    features['melspectrum_mean'], features['melspectrum_var'] = melspectrum_feature(signal)
    features['db_melspectrum_mean'], features['db_melspectrum_var'] = db_melspectrum_feature(signal)
    features['spectral_cent_mean'], features['spectral_cent_var'] = spectral_cent_feature(signal)
    features['spectral_rolloff_mean'], features['spectral_rolloff_var'] = spectral_rolloff_feature(signal)
    features['zero_crossing_mean'], features['zero_crossing_var'] = zero_crossing_feature(signal)
    return features


csv_path = 'songs_features.csv'

try:
    with open(csv_path, 'w') as file:
        file.truncate(0)
    print(f"Файл '{csv_path}' очищен успешно.")
except Exception as e:
    print(f"Ошибка при очистке файла '{csv_path}': {e}")

# Путь к папке с аудиокомпозициями
songs_dir = 'songs/'

# Список имен файлов в папке songs
file_names = os.listdir(songs_dir)

# Создание пустого списка для хранения словарей признаков
segments_data = []

# Добавление признаков в DataFrame для каждого сегмента каждой композиции

for filename in file_names:
    file_path = os.path.join(songs_dir, filename)
    signal, sr = librosa.load(file_path, sr=sr)
    segments = segment_audio(signal, sr, segment_duration, segment_duration - 5)  # Изменение с перекрытием
    for i, segment in enumerate(segments):
        # Обрезаем тишину в начале и в конце сегмента
        segment, _ = librosa.effects.trim(segment) # эта строка убирается, если тишину необходимо оставить
        features = extract_features(segment)
        segment_data = {'filename': f"{filename[:-4]}_{i+1}", **features}
        segments_data.append(segment_data)

# Создание DataFrame из списка словарей
df = pd.DataFrame(segments_data)

# Вывод DataFrame
print(df)
df.to_csv(csv_path, index=False)


Файл 'songs_features.csv' очищен успешно.
                                        filename  spectrum_mean  spectrum_var  \
0    7 Seconds -- Youssou N'Dour, Neneh Cherry_1       0.009078      0.001058   
1    7 Seconds -- Youssou N'Dour, Neneh Cherry_2       1.532228     26.641769   
2    7 Seconds -- Youssou N'Dour, Neneh Cherry_3       1.652172     30.376503   
3    7 Seconds -- Youssou N'Dour, Neneh Cherry_4       2.186472     34.958107   
4    7 Seconds -- Youssou N'Dour, Neneh Cherry_5       2.133529     49.443848   
..                                           ...            ...           ...   
204             Bag Raiders -- Shooting stars_10       2.832639     34.444614   
205             Bag Raiders -- Shooting stars_11       5.193159     94.942253   
206             Bag Raiders -- Shooting stars_12       6.934866    129.035187   
207             Bag Raiders -- Shooting stars_13       8.448624    131.645828   
208             Bag Raiders -- Shooting stars_14       6.249237    

In [38]:
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/Timbaland_-_Apologize_47972715.mp3'
signal = librosa.load(filename_new, sr=22050)[0] #, duration=20
spectrum_mean, spectrum_var = spectrum_feature(signal)
db_spectrum_mean, db_spectrum_var = db_spectrum_feature(signal)
melspectrum_mean, melspectrum_var = melspectrum_feature(signal)
db_melspectrum_mean, db_melspectrum_var = db_melspectrum_feature(signal)
spectral_cent_mean, spectral_cent_var = spectral_cent_feature(signal)
spectral_rolloff_mean, spectral_rolloff_var = spectral_rolloff_feature(signal)
zero_crossing_mean, zero_crossing_var = zero_crossing_feature(signal)

new_features = [spectrum_mean, spectrum_var, db_spectrum_mean, db_spectrum_var, melspectrum_mean, melspectrum_var, db_melspectrum_mean, db_melspectrum_var, spectral_cent_mean, spectral_cent_var, spectral_rolloff_mean, spectral_rolloff_var, zero_crossing_mean, zero_crossing_var]
new_features = np.array(new_features).reshape(1, -1)



X = df.drop(columns=['filename'])
print(X)
print('Features:')
print(new_features)

    spectrum_feature_mean  spectrum_feature_var  db_spectrum_feature_mean  \
0            1.415024e-03          5.448380e-06                 -8.287863   
1            8.429581e-04          8.048017e-06                 -3.834793   
2            7.433493e-20          1.894931e-39                 -7.576152   
3            8.187937e-06          3.205970e-11                 -6.590091   
4            6.735864e-02          2.151134e-01                 -2.964818   
5            1.317450e+00          2.668646e+00                 -5.605087   
6            2.534112e+00          8.626266e+01                 -9.704225   
7            5.115468e-01          5.532359e+00                 -6.469716   
8            1.043127e+00          3.236451e+01                -14.959120   
9            1.066156e-01          1.368918e-01                 -2.647291   
10           1.173418e+00          1.380086e+01                 -1.725452   
11           1.118220e-01          5.227377e-02                 -5.999099   

In [39]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])


Признаки новой композиции:
[[ 7.47407079e-01  1.84110832e+01 -6.43032694e+00  2.03364059e+02
   2.56333947e+00  7.91432434e+02 -3.96839180e+01  2.15656616e+02
   2.26788371e+03  1.19863457e+06  4.96018372e+03  5.62950714e+06
   9.03437396e-02  4.40387526e-03]]

Другие похожие файлы и их признаки:

Файл 1: Apologize.mp3
[ 5.11546850e-01  5.53235860e+00 -6.46971560e+00  2.02547070e+02
  2.37524150e+00  8.22804930e+02 -4.07967600e+01  2.20416100e+02
  2.32040490e+03  1.27557061e+06  5.04599911e+03  5.74612117e+06
  9.41517279e-02  4.73722360e-03]
Расстояние: 139706.8267316446

Файл 2: Appletree.mp3
[ 1.04312700e+00  3.23645060e+01 -1.49591200e+01  1.76028520e+02
  2.99068900e-01  1.67233090e+03 -4.66821670e+01  2.43737230e+02
  1.98215141e+03  1.19390030e+06  4.26731192e+03  5.23577064e+06
  6.29451775e-02  3.57126172e-03]
Расстояние: 393766.6620908847

Файл 3: Animals — Martin Garrix.mp3
[ 1.31745000e+00  2.66864560e+00 -5.60508730e+00  1.99456090e+02
  1.68426120e+00  6.99968440e+02 -3.

In [ ]:
# Создайте словарь для хранения сумм расстояний по композициям
sum_distances = {}

# Проход по каждой строке DataFrame X (признаки каждого фрагмента)
for index, row in df.iterrows():
    # Преобразуйте индекс в строку и получите название композиции из индекса строки
    composition_name = row['filename'].split('_')[0]
    
    # Если композиция еще не добавлена в словарь, добавьте ее
    if composition_name not in sum_distances:
        sum_distances[composition_name] = 0
    
    # Вычислите расстояние между текущим фрагментом и новой композицией
    distance = np.linalg.norm(X.values - new_features)
    
    # Добавьте расстояние к сумме расстояний для данной композиции
    sum_distances[composition_name] += distance

# Преобразуйте словарь в DataFrame для удобства обработки
sum_distances_df = pd.DataFrame(list(sum_distances.items()), columns=['Composition', 'Sum Distance'])

# Найдите наиболее похожие композиции на основе сумм расстояний
closest_compositions = sum_distances_df.nsmallest(11, 'Sum Distance')

# Выведите результаты
print("Наиболее похожие композиции:")
for index, row in closest_compositions.iterrows():
    print(f"{row['Composition']}, Сумма расстояний: {row['Sum Distance']}")


Наиболее похожие композиции:
Apologize, Сумма расстояний: 401798842.77883863
B.o.B, Hayley Williams of Paramore -- Airplanes (feat. Hayley Williams of Paramore), Сумма расстояний: 401798842.77883863
ABBA -- Money, Money, Money, Сумма расстояний: 438326010.3041876
All The Things She Said, Сумма расстояний: 474853177.82953656
Another One Bites The Dust — Queen, Сумма расстояний: 474853177.82953656
Bad Bad Boys, Сумма расстояний: 474853177.82953656
a-ha -- Take On Me, Сумма расстояний: 511380345.3548855
B.o.B, Jessie J -- Price Tag, Сумма расстояний: 511380345.3548855
Bag Raiders -- Shooting stars, Сумма расстояний: 511380345.3548855
Appletree, Сумма расстояний: 620961847.9309325
ABBA -- The Winner Takes It All, Сумма расстояний: 694016182.9816306


### Загрузка файла

In [32]:
# Вычислите расстояния между наборами признаков
distances = np.linalg.norm(X.values - new_features, axis=1)

# Найдите индексы строк с минимальными расстояниями (включая самый близкий)
closest_indices = np.argsort(distances)[:11]  # Выбираем первые четыре индекса с наименьшими расстояниями

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Расстояние:", distances[closest_indices[i]])

Признаки новой композиции:
[[-2.79062164e+02  9.81874771e+01  1.55698334e+02  5.44654579e+01
   1.57122917e+01  3.68546371e+01  3.23547440e+01  3.17071018e+01
   7.65785456e+00  1.68530464e+01  1.50993824e+01  1.59104233e+01
   4.79303741e+00  1.04169302e+01  5.60571718e+00  1.47617292e+01
   1.71552563e+00  1.23163395e+01 -9.36045706e-01  1.00283155e+01
  -1.95086256e-01  8.04045963e+00 -1.35133984e-02  7.22135305e+00
   4.10704470e+00  6.39779472e+00]]

Другие похожие файлы и их признаки:

Файл 1: Appletree_4
[-248.38965     84.76933    122.98244     35.87223     14.906594
   36.474186    53.738476    24.243736     8.060009    24.665209
   20.290806    18.969952    -9.005632    18.11949      7.1829386
   13.994774    -0.647049    11.630134    -2.3296106   12.699757
    5.143558    12.444908     1.7114475   10.630182     1.666215
   10.089953 ]
Расстояние: 59.14537974782552

Файл 2: Appletree_3
[-257.85806      83.774315    112.55922      38.80696      16.265879
   33.167545     55.39

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
# Загрузка данных
df = pd.read_csv('songs_features.csv')
filename_new = 'test_data/7 Seconds from middle small.mp3'
result = process_audio(filename_new, 20, fragment_length=20, overlap=5)
result = list(result[0].values())[1:]
new_features = np.array(result).reshape(1, -1)

X = df.drop(columns=['filename'])

# Вычислите косинусное сходство между наборами признаков
similarities = cosine_similarity(X, new_features)

# Найдите индексы строк с максимальным косинусным сходством (включая самый близкий)
closest_indices = np.argsort(similarities, axis=0)[-11:].flatten()

# Получите названия файлов из найденных строк
closest_files = df.iloc[closest_indices]['filename']

# Получите признаки из найденных строк
closest_features = X.iloc[closest_indices].values

print("Признаки новой композиции:")
print(new_features)

print("\nДругие похожие файлы и их признаки:")
for i in range(len(closest_indices)):
    print(f"\nФайл {i+1}: {closest_files.iloc[i]}")
    print(closest_features[i])
    print("Косинусное сходство:", similarities[closest_indices[i]][0])

Признаки новой композиции:
[[-136.94652     48.15086    133.78339     25.045727    -3.4574358
    19.012459    52.906593    12.004604    -9.7661295   17.015518
    45.452053    11.971169   -22.336512    13.349935    21.156645
    10.17392     -1.8026047   10.184543    13.6172285    8.353845
    -8.874034     8.900497    11.079862     7.5026326    1.893026
     6.3978343]]

Другие похожие файлы и их признаки:

Файл 1: Bad Bad Boys_5
[-150.28946     50.68178    122.11754     22.383467   -18.054884
   29.379808    59.9512      17.397646   -11.749435    21.948305
   31.130291    16.501328   -17.769829    13.568505    26.109
   11.274147    -9.232224     9.664581    13.4331665   11.080587
    1.7540551   10.337084     6.979301     8.990133     4.5127983
    6.9244866]
Косинусное сходство: 0.9875544754059715

Файл 2: Bad Bad Boys_7
[-152.31073     51.82261    117.270546    21.034023   -16.460344
   26.94437     65.02995     15.612344   -10.527432    20.826616
   35.888798    15.061704   -17.